# Drivers

In [3]:
import pandas as pd
import fastf1 as f1
import requests
import os

base_url = "https://api.jolpi.ca/ergast/f1"

season = 2025
event = 'MONACO'
session_type = 'R'

In [7]:
url = f"{base_url}/{season}/drivers.json"
response = requests.get(url)
data = response.json()

drivers = data['MRData']['DriverTable']['Drivers']

rows = []
for driver in drivers:
    rows.append({
        "first_name": driver["givenName"],
        "last_name": driver["familyName"],
        "driver": driver["code"],
        "car_number": driver["permanentNumber"],
        "nationality": driver["nationality"]
    })

df = pd.DataFrame(rows)
df

,first_name,last_name,driver,car_number,nationality
0,Alexander,Albon,ALB,23,Thai
1,Fernando,Alonso,ALO,14,Spanish
2,Andrea Kimi,Antonelli,ANT,12,Italian
3,Oliver,Bearman,BEA,87,British
4,Gabriel,Bortoleto,BOR,5,Brazilian
5,Franco,Colapinto,COL,43,Argentine
6,Jack,Doohan,DOO,7,Australian
7,Pierre,Gasly,GAS,10,French
8,Isack,Hadjar,HAD,6,French
9,Lewis,Hamilton,HAM,44,British


# Constructors

In [8]:
url = f"{base_url}/{season}/constructors.json"
response = requests.get(url)
data = response.json()

constructors = data['MRData']['ConstructorTable']['Constructors']

rows = []
for constructor in constructors:
    rows.append({
        "constructor": constructor["name"],
        "nationality": constructor["nationality"]
    })

df = pd.DataFrame(rows)
df

,constructor,nationality
0,Alpine F1 Team,French
1,Aston Martin,British
2,Ferrari,Italian
3,Haas F1 Team,American
4,McLaren,British
5,Mercedes,German
6,RB F1 Team,Italian
7,Red Bull,Austrian
8,Sauber,Swiss
9,Williams,British


# Standings

In [9]:
url = f"{base_url}/{season}/driverstandings.json"
response = requests.get(url)
data = response.json()

standings = data['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']

rows = []
for driver in standings:
    rows.append({
        "season": season,
        "driver": driver["Driver"]["code"],
        "car_number": driver["Driver"]["permanentNumber"],
        "constructor": driver["Constructors"][0]["name"],
        "position": driver["position"],
        "points": driver["points"],
        "wins": driver["wins"],
    })

df = pd.DataFrame(rows)
df

,season,driver,car_number,constructor,position,points,wins
0,2025,NOR,4,McLaren,1,390,7
1,2025,PIA,81,McLaren,2,366,7
2,2025,VER,33,Red Bull,3,341,5
3,2025,RUS,63,Mercedes,4,276,2
4,2025,LEC,16,Ferrari,5,214,0
5,2025,HAM,44,Ferrari,6,148,0
6,2025,ANT,12,Mercedes,7,122,0
7,2025,ALB,23,Williams,8,73,0
8,2025,HUL,27,Sauber,9,43,0
9,2025,HAD,6,RB F1 Team,10,43,0


In [5]:
url = f"{base_url}/{season}/constructorstandings.json"
response = requests.get(url)
data = response.json()

standings = data['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']

rows = []
for constructor in standings:
    rows.append({
        "season": season,
        "constructor": constructor["Constructor"]["name"],
        "position": constructor["position"],
        "points": constructor["points"],
        "wins": constructor["wins"],
    })

df = pd.DataFrame(rows)
df

,season,constructor,position,points,wins
0,2025,McLaren,1,756,14
1,2025,Mercedes,2,398,2
2,2025,Red Bull,3,366,5
3,2025,Ferrari,4,362,0
4,2025,Williams,5,111,0
5,2025,RB F1 Team,6,82,0
6,2025,Aston Martin,7,72,0
7,2025,Haas F1 Team,8,70,0
8,2025,Sauber,9,62,0
9,2025,Alpine F1 Team,10,22,0


# Schedule

In [11]:
# schedule = f1.get_event_schedule(season)
# rows = []

# for _, event in schedule.iterrows():
#         session = f1.get_session(season, event['EventName'], 'R')
#         session.load()

#         rows.append({
#             "season": season,
#             "round": event['RoundNumber'],
#             "event": event['EventName'],
#             "country": event['Country'],
#             "location": event['Location'],
#             "date": event['EventDate'],
#             "format": event['EventFormat'],
#             "race_laps": session.total_laps,
#         })

# df = pd.DataFrame(rows)
# df

# Session Results

In [4]:
session = f1.get_session(season, event, session_type)
session.load()
df = session.results.copy()

if session_type in ['R', 'S']:
    df = df[['DriverNumber', 'Abbreviation', 'TeamName', 'Position', 'GridPosition', 'Status']]
    df.rename(columns={
        'DriverNumber': 'car_number',
        'Abbreviation': 'driver',
        'TeamName': 'constructor',
        'Position': 'finish_pos',
        'GridPosition': 'grid_pos',
        'Status': 'status'
    }, inplace=True)

elif session_type in ['Q', 'SQ', 'SS']:
    df = df[['DriverNumber', 'Abbreviation', 'TeamName', 'Q1', 'Q2', 'Q3', 'Position']]
    df.rename(columns={
        'DriverNumber': 'car_number',
        'Abbreviation': 'driver',
        'TeamName': 'constructor',
        'Position': 'finish_pos'
    }, inplace=True)

df['season'] = season
df['event'] = event
df['session_type'] = session_type

df

req         WARNING 	DEFAULT CACHE ENABLED! (7.98 MB) C:\Users\JOHN AZIZ\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Monaco Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	No cached data found for car_data. Loading data...
_api           INFO 	Fetching car data...
_api           INFO 	Parsing car data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
_api           INFO 	Parsing position 

,car_number,driver,constructor,finish_pos,grid_pos,status,season,event,session_type
4,4,NOR,McLaren,1.0,1.0,Finished,2025,MONACO,R
16,16,LEC,Ferrari,2.0,2.0,Finished,2025,MONACO,R
81,81,PIA,McLaren,3.0,3.0,Finished,2025,MONACO,R
1,1,VER,Red Bull Racing,4.0,4.0,Finished,2025,MONACO,R
44,44,HAM,Ferrari,5.0,7.0,Finished,2025,MONACO,R
6,6,HAD,Racing Bulls,6.0,5.0,Lapped,2025,MONACO,R
31,31,OCO,Haas F1 Team,7.0,8.0,Lapped,2025,MONACO,R
30,30,LAW,Racing Bulls,8.0,9.0,Lapped,2025,MONACO,R
23,23,ALB,Williams,9.0,10.0,Lapped,2025,MONACO,R
55,55,SAI,Williams,10.0,11.0,Lapped,2025,MONACO,R


# Laptimes

In [6]:
session = f1.get_session(season, event, 'R')
session.load()
df = session.laps.copy()

df = df[['Driver', 'DriverNumber', 'Team', 'LapNumber', 'LapTime', 'Position', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'Stint', 'Compound']]
df.rename(columns={
    'Driver': 'driver',
    'DriverNumber': 'car_number',
    'Team': 'constructor',
    'LapNumber': 'lap_number',
    'LapTime': 'lap_time',
    'Position': 'position',
    'Sector1Time': 'sector1',
    'Sector2Time': 'sector2',
    'Sector3Time': 'sector3',
    'Stint': 'stint',
    'Compound': 'tyre_compound'
}, inplace=True)

df['season'] = season
df['event'] = event
df['session_type'] = 'R'

df

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '16', '81', '1', '44', '6', '31', '30', '23', '55', '63', '87', '43', '5', '18', '27', '22', '12', '14', '10']


,driver,car_number,constructor,lap_number,lap_time,position,sector1,sector2,sector3,stint,tyre_compound,season,event,session_type
0,VER,1,Red Bull Racing,1.0,0 days 00:01:27.020000,4.0,NaT,0 days 00:00:38.394000,0 days 00:00:21.759000,1.0,HARD,2025,MONACO,R
1,VER,1,Red Bull Racing,2.0,0 days 00:01:46.318000,4.0,0 days 00:00:34.949000,0 days 00:00:45.145000,0 days 00:00:26.224000,1.0,HARD,2025,MONACO,R
2,VER,1,Red Bull Racing,3.0,0 days 00:01:48.815000,4.0,0 days 00:00:32.110000,0 days 00:00:47.430000,0 days 00:00:29.275000,1.0,HARD,2025,MONACO,R
3,VER,1,Red Bull Racing,4.0,0 days 00:01:40.290000,4.0,0 days 00:00:31.006000,0 days 00:00:47.889000,0 days 00:00:21.395000,1.0,HARD,2025,MONACO,R
4,VER,1,Red Bull Racing,5.0,0 days 00:01:20.800000,4.0,0 days 00:00:21.744000,0 days 00:00:37.485000,0 days 00:00:21.571000,1.0,HARD,2025,MONACO,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1420,BEA,87,Haas F1 Team,72.0,0 days 00:01:15.427000,13.0,0 days 00:00:19.724000,0 days 00:00:35.867000,0 days 00:00:19.836000,3.0,MEDIUM,2025,MONACO,R
1421,BEA,87,Haas F1 Team,73.0,0 days 00:01:15.355000,12.0,0 days 00:00:19.631000,0 days 00:00:35.894000,0 days 00:00:19.830000,3.0,MEDIUM,2025,MONACO,R
1422,BEA,87,Haas F1 Team,74.0,0 days 00:01:15.365000,12.0,0 days 00:00:19.528000,0 days 00:00:35.823000,0 days 00:00:20.014000,3.0,MEDIUM,2025,MONACO,R
1423,BEA,87,Haas F1 Team,75.0,0 days 00:01:15.799000,12.0,0 days 00:00:19.650000,0 days 00:00:36.177000,0 days 00:00:19.972000,3.0,MEDIUM,2025,MONACO,R


# Pitstops

In [7]:
race_count = f1.get_event_schedule(season).RoundNumber.max()

stops = []

for rnd in range(1, race_count + 1):
    url = f"{base_url}/{season}/{rnd}/pitstops.json"
    response = requests.get(url)
    data = response.json()

    races = data['MRData']['RaceTable']['Races']

    if not races:
        continue

    race = races[0]

    for stop in race.get('PitStops', []):
        stops.append({
            "season": season,
            "race": race['raceName'],
            "driver": stop["driverId"],
            "stop": stop["stop"],
            "lap_number": stop["lap"],
            "duration": stop["duration"]
        })

df = pd.DataFrame(stops)
df

,season,race,driver,stop,lap_number,duration
0,2025,Australian Grand Prix,norris,1,2,13.341
1,2025,Australian Grand Prix,max_verstappen,1,2,13.416
2,2025,Australian Grand Prix,piastri,1,2,13.078
3,2025,Australian Grand Prix,russell,1,2,13.672
4,2025,Australian Grand Prix,leclerc,1,2,14.182
...,...,...,...,...,...,...
544,2025,São Paulo Grand Prix,bearman,2,42,24.654
545,2025,São Paulo Grand Prix,colapinto,2,43,26.124
546,2025,São Paulo Grand Prix,alonso,2,46,23.563
547,2025,São Paulo Grand Prix,antonelli,2,47,23.401


# Weather Data

In [8]:
session = f1.get_session(season, event, session_type)
session.load()

df = session.weather_data.copy()

df.rename(columns={
    'Time': 'time',
    'AirTemp': 'air_temp_c',
    'TrackTemp': 'track_temp_c',
    'Humidity': 'humidity_pct',
    'Pressure': 'pressure_mbar',
    'Rainfall': 'rainfall',
    'WindSpeed': 'wind_speed_kph',
    'WindDirection': 'wind_dir_deg'
}, inplace=True)

df['season'] = season
df['event'] = event
df['session_type'] = session_type

df

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '16', '81', '1', '44', '6', '31', '30', '23', '55', '63', '87', '43', '5', '18', '27', '22', '12', '14', '10']


,time,air_temp_c,humidity_pct,pressure_mbar,rainfall,track_temp_c,wind_dir_deg,wind_speed_kph,season,event,session_type
0,0 days 00:00:18.586000,21.5,55.0,1017.3,False,42.6,298,1.0,2025,MONACO,R
1,0 days 00:01:18.584000,21.6,54.0,1017.3,False,43.6,176,0.7,2025,MONACO,R
2,0 days 00:02:18.615000,21.6,54.0,1017.3,False,43.5,180,2.2,2025,MONACO,R
3,0 days 00:03:18.615000,21.4,55.0,1017.2,False,43.3,0,1.2,2025,MONACO,R
4,0 days 00:04:18.632000,21.3,55.0,1017.1,False,43.2,341,0.4,2025,MONACO,R
...,...,...,...,...,...,...,...,...,...,...,...
155,0 days 02:35:19.337000,22.3,51.0,1016.2,False,41.3,185,0.9,2025,MONACO,R
156,0 days 02:36:19.337000,22.4,50.0,1016.2,False,41.9,195,1.5,2025,MONACO,R
157,0 days 02:37:19.337000,22.4,50.0,1016.2,False,41.7,264,1.0,2025,MONACO,R
158,0 days 02:38:19.322000,22.4,51.0,1016.2,False,41.4,192,1.5,2025,MONACO,R


# Race Events

In [9]:
session = f1.get_session(season, event, 'R')
session.load()

df = session.race_control_messages.copy()
df = df[['Time', 'Category', 'Message', 'Flag', 'Scope', 'Lap']]

df.rename(columns = {
    'Time': 'time',
    'Category': 'category',
    'Message': 'message',
    'Flag': 'flag',
    'Scope': 'scope',
    'Lap': 'lap_number'
}, inplace=True)

df['season'] = season
df['event'] = event
df['session_type'] = 'R'

df

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '16', '81', '1', '44', '6', '31', '30', '23', '55', '63', '87', '43', '5', '18', '27', '22', '12', '14', '10']


,time,category,message,flag,scope,lap_number,season,event,session_type
0,2025-05-25 12:20:01,Flag,GREEN LIGHT - PIT EXIT OPEN,GREEN,Track,1,2025,MONACO,R
1,2025-05-25 12:30:01,Other,PIT EXIT CLOSED,None,None,1,2025,MONACO,R
2,2025-05-25 12:31:26,Flag,DOUBLE YELLOW IN TRACK SECTOR 7,DOUBLE YELLOW,Sector,1,2025,MONACO,R
3,2025-05-25 12:31:36,Flag,CLEAR IN TRACK SECTOR 7,CLEAR,Sector,1,2025,MONACO,R
4,2025-05-25 12:37:42,Other,INCIDENT INVOLVING CAR 87 (BEA) NOTED - UNSAFE...,None,None,1,2025,MONACO,R
...,...,...,...,...,...,...,...,...,...
191,2025-05-25 14:43:43,Flag,CHEQUERED FLAG,CHEQUERED,Track,78,2025,MONACO,R
192,2025-05-25 14:44:02,Other,TURN 10 INCIDENT INVOLVING CARS 27 (HUL) AND 1...,None,None,78,2025,MONACO,R
193,2025-05-25 14:45:13,Other,FIA STEWARDS: TURN 10 INCIDENT INVOLVING CARS ...,None,None,78,2025,MONACO,R
194,2025-05-25 14:45:40,Other,FIA STEWARDS: PENALTY SERVED - DRIVE THROUGH P...,None,None,78,2025,MONACO,R
